In [1]:
import pandas as pd
import numpy as np

np.random.seed(1349)

In [4]:
df = pd.read_csv('students.csv', index_col=0)

## Reshaping

We will talk about reshaping operations in more detail when we discuss tidy data, but for now we will focus on a couple of common operations that can be used to summarize our data by different subgroups.

### `pd.crosstab`

For an example of `.crosstab`, we will count the number of students passing math in each classroom.

In [5]:
df.head()

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A


In [7]:
df['passing_math'] = np.where(df.math >=70, 'passing', 'failing')

In [8]:
df.passing_math

0     failing
1     passing
2     passing
3     passing
4     passing
5     passing
6     passing
7     passing
8     passing
9     failing
10    passing
11    passing
Name: passing_math, dtype: object

In [6]:
df.classroom

0     A
1     B
2     A
3     B
4     A
5     B
6     A
7     A
8     A
9     A
10    B
11    A
Name: classroom, dtype: object

In [9]:
df.classroom.value_counts()

A    8
B    4
Name: classroom, dtype: int64

In [10]:
df.passing_math.value_counts()

passing    10
failing     2
Name: passing_math, dtype: int64

In [ ]:
#  cant do this : df.crosstab(df.passing_math, df.classroom)

In [11]:
# We will use our student grades DataFrame, df.
pd.crosstab(df.classroom, df.passing_math)


passing_math,failing,passing
classroom,,
A,2,6
B,0,4


We'll use the `pd.crosstab` function to count the number of occurances of each subgroup (i.e. each unique combination of classroom and whether or not the student is passing math):

We can also view subtotals with the `margins` set to `True`.

In [12]:
pd.crosstab(df.classroom, df.passing_math, margins=True)


passing_math,failing,passing,All
classroom,,,
A,2,6,8
B,0,4,4
All,2,10,12


The `.crosstab` function will let us view the numbers as percentages of the total as well by setting `normalize` to `True`.

In [13]:
pd.crosstab(df.classroom, df.passing_math, margins=True, normalize=True)

passing_math,failing,passing,All
classroom,,,
A,0.166667,0.500000,0.666667
B,0.000000,0.333333,0.333333
All,0.166667,0.833333,1.000000


In [14]:
# crosstab will give you everything counted between the intersection of two different
# categorical variables in a dataframe

### `.pivot_table`

Here we use the `.pivot_table` method to create our summary. This method produces output similar to an excel pivot table. We must supply 3 things here:

- which values will make up the rows (the `index`)
- which values will make up the columns
- the values we are aggregating
- an aggregation method (`aggfunc`); if we can omit this, and `mean` will be used by default

For an example using the `pivot_table` method, we'll calculate the average math grade for the combination of `classroom` and `passing_math` status.

In [16]:
pd.DataFrame.pivot_table?

Signature:
pd.DataFrame.pivot_table(
    self,
    values=None,
    index=None,
    columns=None,
    aggfunc='mean',
    fill_value=None,
    margins=False,
    dropna=True,
    margins_name='All',
    observed=False,
) -> 'DataFrame'
Docstring:
Create a spreadsheet-style pivot table as a DataFrame.

The levels in the pivot table will be stored in MultiIndex objects
(hierarchical indexes) on the index and columns of the result DataFrame.

Parameters
----------
values : column to aggregate, optional
index : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types (except list).
    Keys to group by on the pivot table index.  If an array is passed,
    it is being used as the same manner as column values.
columns : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types (except

In [17]:
df.pivot_table(index='classroom', columns='passing_math', values='math', aggfunc='max')

passing_math,failing,passing
classroom,,
A,69.0,94.0
B,NaN,98.0


In [18]:
df.groupby(['passing_math', 'classroom']).math.max()

passing_math  classroom
failing       A            69
passing       A            94
              B            98
Name: math, dtype: int64

Here we'll create a dataframe that represents various orders at a restaurant.

In [19]:
n = 40

orders = pd.DataFrame({
    'drink': np.random.choice(['Tea', 'Water', 'Water'], n),
    'meal': np.random.choice(['Curry', 'Yakisoba Noodle', 'Pad Thai'], n),
})
# .sample will give me n number of random rows
orders.sample(10)

,drink,meal
26,Water,Yakisoba Noodle
19,Water,Curry
17,Water,Curry
27,Water,Curry
7,Water,Pad Thai
2,Water,Pad Thai
29,Water,Pad Thai
33,Water,Yakisoba Noodle
37,Tea,Yakisoba Noodle
14,Tea,Yakisoba Noodle


#### `.map`

The `.map` method lets us use a dictionary to calculate the total price for an order; then I can save my calculations to a new column named `bill`. Let's do this step-by-step.

In [20]:
# Create a dictionary of prices for drinks and meals.

prices = {
    'Yakisoba Noodle': 9,
    'Curry': 11,
    'Pad Thai': 10,
    'Tea': 2,
    'Water': 0,
}

In [21]:
"""
Match the values in the 'drink' and 'meal' columns with the values in the 'prices' dictionary 
and perform the specified calculation. Save this calculation to a new column named 'bill'.
"""
# the dataframe we want to reference
# the column we want to reference,
# what we want to do with that Series (map)
# from what reference are we mapping? (prices)
orders['bill'] = orders.drink.map(prices) + orders.meal.map(prices)

In [22]:
orders.head()

,drink,meal,bill
0,Water,Pad Thai,10
1,Water,Curry,11
2,Water,Pad Thai,10
3,Tea,Pad Thai,12
4,Water,Pad Thai,10


Let's take a look at how many orders have each combination of meal and drink:

In [24]:
pd.crosstab(orders.drink, orders.meal, margins=True)

meal,Curry,Pad Thai,Yakisoba Noodle,All
drink,,,,
Tea,2,5,4,11
Water,9,11,9,29
All,11,16,13,40


In [25]:
orders.pivot_table(index='drink', columns='meal', values='bill', aggfunc='mean')

meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,13,12,11
Water,11,10,9


And let's find out the average bill amount for each combination: 

It's interesting to note that we could find the same information with a multi-level group by:

In [27]:
orders.groupby(['meal', 'drink']).bill.agg('max')

meal             drink
Curry            Tea      13
                 Water    11
Pad Thai         Tea      12
                 Water    10
Yakisoba Noodle  Tea      11
                 Water     9
Name: bill, dtype: int64

The choice between group by and a pivot table here is mostly asthetic, and you should use whichever makes more sense to you with the problem at hand. 

### Transposing

In [29]:
orders.T

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
drink,Water,Water,Water,Tea,Water,Water,Water,Water,Water,Water,...,Tea,Tea,Tea,Water,Water,Water,Water,Tea,Water,Water
meal,Pad Thai,Curry,Pad Thai,Pad Thai,Pad Thai,Pad Thai,Yakisoba Noodle,Pad Thai,Pad Thai,Pad Thai,...,Pad Thai,Pad Thai,Pad Thai,Yakisoba Noodle,Curry,Yakisoba Noodle,Yakisoba Noodle,Yakisoba Noodle,Pad Thai,Yakisoba Noodle
bill,10,11,10,12,10,10,9,10,10,10,...,12,12,12,9,11,9,9,11,10,9


In [31]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
math,12.0,84.833333,11.134168,62.0,78.50,90.0,92.25,98.0
english,12.0,77.666667,13.371158,62.0,63.75,77.5,86.75,99.0
reading,12.0,86.500000,9.643651,67.0,80.75,89.0,93.25,98.0


## Exercises III

1. Use your `get_db_url` function to help you explore the data from the `chipotle` database. 

2. What is the total price for each order?

3. What are the most popular 3 items?

4. Which item has produced the most revenue?

5. Using the `titles` DataFrame, visualize the number of employees with each title.

6. Join the `employees` and `titles` DataFrames together.

7. Visualize how frequently employees change titles.

8. For each title, find the hire date of the employee that was hired most recently with that title.

9. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)